# Dataset


## Generator of dataset into txt file


In [ ]:
# Force to overwritten txt file
# True -> overwritten

force = True

In [ ]:
import random
import os

def create_dataset_file(num_customers):

  # Generate random time windows.
  time_windows = []
  for i in range(num_customers+1):
    start_time = random.randint(0, 30)
    end_time = random.randint(start_time+20, 100)
    time_windows.append([start_time, end_time])

  # Create a symmetric time matrix.
  num_matrix = num_customers+1
  time_matrix = [[0 for i in range(num_matrix)] for j in range(num_matrix)]
  for i in range(num_matrix):
    for j in range(num_matrix):
      if i == j:
        time_matrix[i][j] = 0
      elif i < j:
        time_matrix[i][j] = abs(random.randint(1,10))
      else:
        time_matrix[i][j] = time_matrix[j][i]

  # Create the .txt file.
  file_contents = f"#N => number of customers\n{num_customers}\n\n"
  file_contents += "#[ai, bi] => Time windows\n"
  time_windows.pop(-1)
  for time_window in time_windows:
    file_contents += f"{time_window[0]} {time_window[1]}\n"

  file_contents += "\n#T => time matrix\n"
  for row in time_matrix:
    file_contents += " ".join([str(element) for element in row]) + "\n"

  return file_contents


num_customers = 5  # <-----------------------------------------------

# Create the dataset
file_contents = create_dataset_file(num_customers=num_customers)
name_file = f"Dataset_{num_customers}.txt"

# Write the file contents to a .txt file, if it not already exist
if not os.path.exists(name_file) or force:
  if os.path.exists(name_file):
    print(name_file, " found and it will BE overwritten.")
  else:
    print(name_file, " generated.")
  with open(name_file, "w") as f:
    f.write(file_contents)
else:
  print(name_file, " found and it will NOT BE overwritten.")

Dataset_5.txt  generated.


## Load data from txt instance

In [ ]:
# Define filename to be read
name_file = f"Dataset_{num_customers}.txt"

# For use a specific dataset
#name_file = 'tsp-tw_instance_5.txt'
#num_customers = 5

In [ ]:
with open(name_file, "r") as f:
  lines = f.readlines()

  # Total numbers of customers
  N = int(lines[1])

  # Time windows
  placeholder = 4
  max_line = placeholder + N

  time_windows = []
  temp = []

  for i in range(placeholder, max_line):
   temp.append(lines[i].split())

  for i in temp:
    ai ,bi = i
    ai, bi = int(ai), int(bi)
    time_windows.append((ai,bi))

  # Cost matrix
  placeholder = 4+N+2
  max_line = placeholder + N +1 #Add depot

  cost_matrix = []

  for i in range(placeholder, max_line):

    temp2 = []

    for num in lines[i].split():
      temp2.append(int(num))

    cost_matrix.append(temp2)

In [ ]:
# Printing of read file
print('Number of customers: ',N)
print('--------------------------')
print('Time windows for each customer:')
for i in time_windows:
  print(i)
print('--------------------------')
print('Cost matrix:')
for i in cost_matrix:
  print(i)

Number of customers:  5
--------------------------
Time windows for each customer:
(5, 80)
(19, 81)
(16, 64)
(23, 44)
(26, 99)
--------------------------
Cost matrix:
[0, 9, 10, 7, 4, 3]
[9, 0, 8, 9, 8, 1]
[10, 8, 0, 1, 8, 3]
[7, 9, 1, 0, 3, 6]
[4, 8, 8, 3, 0, 6]
[3, 1, 3, 6, 6, 0]


# PuLP Implementation

## Usefull function

In [ ]:
# Function that store messages and print it
def print_and_store(message, only_store=False):
    if not only_store:
      print(message)
    messages.append(message)

In [ ]:
# Function that create a log file
def save_optimization_results(number_customers, name_heuristic, status, objective_value, computing_time, messages):
    variables = "\n".join(messages)

    results_str = (
        f"Instance Name: {name_heuristic}\n"
        f"Number of Clients: {number_customers}\n"
        f"Status: {status}\n"
        f"Objective Function Value: {objective_value if objective_value is not None else 'N/A'}\n"
        f"Computing Time: {computing_time} seconds\n"
        f"Variables' Values:\n"
        f"------------------------------------------------\n"
        f"\n{variables if variables else 'N/A'}"
    )

    file_name = f"{name_heuristic}_{number_customers}.txt"

    with open(file_name, 'w') as file:
        file.write(results_str)

In [ ]:
# Print tour

def printTour(tour_vars):

  tour = ['0']
  temp_list = tour_vars.copy()

  while temp_list:

    for val in temp_list:
      # Get the starting and ending points of the tour node.
      start_point = val.split("_")[1]
      end_point = val.split("_")[-1]

      # Add the starting point to the tour.
      if start_point == tour[-1]:
        tour.append(end_point)
        temp_list.remove(val)

      #print(f'temp_list= {temp_list} , tour= {tour}')   # Uncomment for showing the process

  # Print tour
  to_print =''
  for i in tour:
    to_print = to_print + str(i) + ' -> '
  to_print = to_print + "Finished!"

  return to_print

In [ ]:
!pip install pulp
!pip3 install pulp

from pulp import *
import time

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 36.9 MB/s eta 0:00:00


## Pulp solver

In [ ]:
messages = []

# Model
tsptw = LpProblem (name="TSP", sense=1)

# Creating variables
index = range(N+1)  #customers + depot
index2 =range(N)    #customers
M = 1000000         #big constant

x = LpVariable.dicts("x", (index, index), cat="Binary")
beta = LpVariable.dicts("beta", (index), lowBound=0)

# Objective function
tsptw += lpSum (cost_matrix[i][j]*x[i][j] for i in index for j in index), "Objective"

# Constraints
for j in index:
  tsptw += lpSum (x[i][j] for i in index if i!= j)==1, f"constraint-1 | {j}"

for i in index:
  tsptw += lpSum (x[i][j] for j in index if i!= j)==1, f"constraint-2 | {i}"

for j in index:
  if j != 0:
    for i in index:
      tsptw += beta[j] >= beta[i] + cost_matrix[i][j] - M*(1-x[i][j]), f"constraint-3 | {i} {j}"

for i in index2:
  tsptw += time_windows[i][0] <= beta[i] <= time_windows[i][1], f"constraint-4 | {i}"

# Create file
tsptw.writeLP(f"TSP_{N}.lp")

# Solve the problem
tic = time.perf_counter()
tsptw.solve()
toc = time.perf_counter()

# Print statement
if tsptw.status == 1:
  print(f'Model status: {tsptw.status} -> Solved \n')
else:
  raise ValueError(f'Model status: {tsptw.status} It\'s a PROBLEMS!!')

tsp_pulp_value = tsptw.objective.value()
tsp_pulp_time = toc-tic

print(f"objective value: {tsp_pulp_value}\n")

print(f"total time: {round(tsp_pulp_time,3)} seconds\n")


# Print all variables
tour_var = []

for var in tsptw.variables():
  print_and_store(f'variable {var.name} has value: {var.value()}', True)
  if (var.value() >= 0.9) and (var.name[0] == 'x'):
    tour_var.append(var.name)

# Get Pulp tour
tsp_pulp_tour = printTour(tour_var)
print_and_store(f'\nTSP tour: {tsp_pulp_tour}\n')

Model status: 1 -> Solved 

objective value: 20.0

total time: 0.052 seconds


TSP tour: 0 -> 4 -> 3 -> 2 -> 1 -> 5 -> 0 -> Finished!



In [ ]:
save_optimization_results(number_customers=N,
                          name_heuristic='Pulp',
                          status=tsptw.status,
                          objective_value=tsp_pulp_value,
                          computing_time=tsp_pulp_time,
                          messages=messages)


# Heuristic

## Useful functions



In [ ]:
!pip install networkx

import networkx as nx

In [ ]:
def a_star(nodes_list, cost_matrix, start, end):

  # Create a feaseble path matrix
  permutations = list(itertools.permutations(nodes_list, 2))
  new_matrix = [[0 for u in range(len(cost_matrix))] for u in range(len(cost_matrix))]

  for (i, j) in permutations:
    new_matrix[i][j] = cost_matrix[i][j]

  have_result = True

  # Use A* algorithm
  G = nx.Graph()

  for i in range(len(new_matrix)):
      for j in range(i + 1, len(new_matrix)):
          if new_matrix[i][j] > 0:
              G.add_edge(i, j, weight=new_matrix[i][j])

  try:
    path = nx.shortest_path(G, source=start, target=end, weight='weight')
    cost = sum(G[u][v]['weight'] for u, v in zip(path[:-1], path[1:]))
  except:
    have_result = False

  if have_result:
    return(have_result, path, cost)
  else:
    return(have_result, 0, 0)

In [ ]:
def check_open_client(sorted_time_windows_dict, beta):
  keys_within_beta = [key for key, (start, end) in sorted_time_windows_dict.items() if start <= beta <= end]
  return keys_within_beta

In [ ]:
def update_temp(temp_list, cost, list_of_passage):
  temp_list[0] = cost
  temp_list[1] = list_of_passage

In [ ]:
def update_list(temp, sorted_time_windows_dict, tsp_tour):

  # Update tour list
  if tsp_tour[-1] == temp[1][0]:
    tsp_tour.extend(temp[1][1:])
  else:
    raise ValueError("Percorso non compatibile")

  # Clear sorted_time_windows_dict from customers visited
  for deleting in tsp_tour:
      if deleting in sorted_time_windows_dict:
          del sorted_time_windows_dict[deleting]

## Heuristic with A*

In [ ]:
messages = []
status_H1=1

T = 0
delta = max((sum([cost for row in cost_matrix for cost in row])//(len(cost_matrix)**2))//2,2)
beta_H1 = 0

custumers_to_visit = {i + 1: window for i, window in enumerate(time_windows)}
sorted_time_windows_list = sorted(custumers_to_visit.items(), key=lambda item: item[1][1])
sorted_time_windows_dict = dict(sorted_time_windows_list)

tsp_H1_tour = [0]

a_star_times = 0
direct_times = 0

tic = time.perf_counter()

while sorted_time_windows_dict:

  temp = [0, list()]  #cost and list of passage
  chiave = list(sorted_time_windows_dict.keys())
  target = chiave[T]
  print_and_store(f'\nActual max delay: {delta}')
  print_and_store(f'Actual T = {T}')
  print_and_store(f'Actial beta = {beta_H1}')
  print_and_store(f'Actual tour list = {tsp_H1_tour}')
  print_and_store(f'Costumer to visit ordered: {list(sorted_time_windows_dict.keys())}')
  print_and_store(f'\nJournet from {tsp_H1_tour[-1]} to {target}\n')

  # Direct cost
  direct_cost = cost_matrix[tsp_H1_tour[-1]][target]

  # Indirect cost
  node_list = check_open_client(sorted_time_windows_dict, beta_H1)
  print_and_store(f'Actual open client = {node_list}')
  node_list.append(tsp_H1_tour[-1])
  node_list.append(target)

  indirect_cost = 100000
  check_indirect = a_star(node_list, cost_matrix, tsp_H1_tour[-1], target)
  if check_indirect[0]:
    indirect_cost = check_indirect[2]
    print_and_store('Exist a A* path')

  # Better path
  if indirect_cost < direct_cost:
    update_temp(temp, indirect_cost, check_indirect[1])
    a_star_times = a_star_times +1
    print_and_store(f'The best path is A*, indirect path, and pass throw {temp[1]}, at cost: {temp[0]}')
  elif (indirect_cost == direct_cost) and len(check_indirect[1]) == 2:
    update_temp(temp, direct_cost, [tsp_H1_tour[-1], target])
    direct_times = direct_times+1
    print_and_store(f'A* is equal in cost to direct path, and pass throw {temp[1]}, at cost: {temp[0]}')
  elif (indirect_cost == direct_cost) and len(check_indirect[1]) > 2:
    update_temp(temp, indirect_cost, check_indirect[1])
    a_star_times = a_star_times +1
    print_and_store(f'A* is equal in cost to direct path but pass throw more element: {temp[1]}, at cost: {temp[0]}')
  else:
    update_temp(temp, direct_cost, [tsp_H1_tour[-1], target])
    direct_times = direct_times+1
    print_and_store(f'The best path is direct, and pass throw {temp[1]}, at cost: {temp[0]}')

  print_and_store(f'\nbeta_H1 at position {tsp_H1_tour[-1]} was {beta_H1} and beta_H1 at position {target} is {beta_H1+temp[0]}')
  print_and_store(f'Time windows: ai = {sorted_time_windows_dict[target][0]}, bi = {sorted_time_windows_dict[target][1]}')

  # Check arrive-windows
  if sorted_time_windows_dict[target][0] <= beta_H1+temp[0] <= sorted_time_windows_dict[target][1]:
    T=0
    beta_H1 = beta_H1 + temp[0]
    update_list(temp, sorted_time_windows_dict, tsp_H1_tour)
    print_and_store(f'Arrived in time, tour list = {tsp_H1_tour} and beta_H1 = {beta_H1}')
  elif not(sorted_time_windows_dict[target][0] <= beta_H1+temp[0]) and (sorted_time_windows_dict[target][0]-delta <= beta_H1+temp[0] <= sorted_time_windows_dict[target][1]):
    T=0
    delay =  sorted_time_windows_dict[target][0] - (beta_H1+temp[0])
    beta_H1 = beta_H1 + delay + temp[0]
    update_list(temp, sorted_time_windows_dict, tsp_H1_tour)
    print_and_store(f'Arriced in advance and WAIT, tour list = {tsp_H1_tour} and beta_H1 = {beta_H1} with delay: {delay}')
  elif not(sorted_time_windows_dict[target][0] <= beta_H1+temp[0]):
    print_and_store('Too early, go to nwxt costumer is queue')
    T = T +1
    if T >= len(sorted_time_windows_dict):
      delta = delta + delta//2
      T = 0
  else:
    print_and_store('Infeasible situation encountered.------------------------------------------------------------------')
    status_H1 = 0
    break

  print_and_store('\n-----------------------------------------------------------------------------------------------')

beta_H1 = beta_H1 + cost_matrix[tsp_H1_tour[-1]][0]
tsp_H1_tour.append(0)

toc = time.perf_counter()
tsp_H1_time = toc-tic

all_times = a_star_times + direct_times

print_and_store('\nEND\n')

save_optimization_results(number_customers=N,
                          name_heuristic='Heuristic with A*',
                          status=status_H1,
                          objective_value=beta_H1,
                          computing_time=tsp_H1_time,
                          messages=messages)

if status_H1:
  print(f'Status: {status_H1}')
else:
  print(("Infisible, result saved in txt file"))
  raise ValueError("Infisible, result saved in txt file")


Actual max delay: 2
Actual T = 0
Actial beta = 0
Actual tour list = [0]
Costumer to visit ordered: [4, 3, 1, 2, 5]

Journet from 0 to 4

Actual open client = []
Exist a A* path
A* is equal in cost to direct path, and pass throw [0, 4], at cost: 4

beta_H1 at position 0 was 0 and beta_H1 at position 4 is 4
Time windows: ai = 23, bi = 44
Too early, go to nwxt costumer is queue

-----------------------------------------------------------------------------------------------

Actual max delay: 2
Actual T = 1
Actial beta = 0
Actual tour list = [0]
Costumer to visit ordered: [4, 3, 1, 2, 5]

Journet from 0 to 3

Actual open client = []
Exist a A* path
A* is equal in cost to direct path, and pass throw [0, 3], at cost: 7

beta_H1 at position 0 was 0 and beta_H1 at position 3 is 7
Time windows: ai = 16, bi = 64
Too early, go to nwxt costumer is queue

-----------------------------------------------------------------------------------------------

Actual max delay: 2
Actual T = 2
Actial beta = 

## Heuristic pure (without A*)

In [ ]:
messages = []
status_H2=1

T = 0
delta = max((sum([cost for row in cost_matrix for cost in row])//(len(cost_matrix)**2))//2,2)
beta_H2 = 0

custumers_to_visit = {i + 1: window for i, window in enumerate(time_windows)}
sorted_time_windows_list = sorted(custumers_to_visit.items(), key=lambda item: item[1][1])
sorted_time_windows_dict = dict(sorted_time_windows_list)

tsp_H2_tour = [0]

counter_without_a = 0

tic = time.perf_counter()

while sorted_time_windows_dict:
  temp = [0, list()]  #cost and list of passage
  chiave = list(sorted_time_windows_dict.keys())
  target = chiave[T]
  print_and_store(f'\nActual max delay: {delta}')
  print_and_store(f'Actual T = {T}')
  print_and_store(f'Actial beta = {beta_H2}')
  print_and_store(f'Actual tour list = {tsp_H2_tour}')
  print_and_store(f'Costumer to visit ordered: {list(sorted_time_windows_dict.keys())}')
  print_and_store(f'\nJournet from {tsp_H2_tour[-1]} to {target}\n')

  # Direct cost
  direct_cost = cost_matrix[tsp_H2_tour[-1]][target]
  update_temp(temp, direct_cost, [tsp_H2_tour[-1], target])
  print_and_store(f'Direct path pass throw {temp[1]}, at cost: {temp[0]}')

  print_and_store(f'\nbeta_H2 at position {tsp_H2_tour[-1]} was {beta_H2} and beta_H2 at position {target} is {beta_H2+temp[0]}')
  print_and_store(f'Time windows: ai = {sorted_time_windows_dict[target][0]}, bi = {sorted_time_windows_dict[target][1]}')

  counter_without_a = counter_without_a+1

  # Check arrive-windows
  if sorted_time_windows_dict[target][0] <= beta_H2+temp[0] <= sorted_time_windows_dict[target][1]:
    T=0
    beta_H2 = beta_H2 + temp[0]
    update_list(temp, sorted_time_windows_dict, tsp_H2_tour)
    print_and_store(f'Arriced in time, tour list = {tsp_H2_tour} and beta_H2 = {beta_H2}')
  elif not(sorted_time_windows_dict[target][0] <= beta_H2+temp[0]) and (sorted_time_windows_dict[target][0]-delta <= beta_H2+temp[0] <= sorted_time_windows_dict[target][1]):
    T=0
    delay =  sorted_time_windows_dict[target][0] - (beta_H2+temp[0])
    beta_H2 = beta_H2 + delay + temp[0]
    update_list(temp, sorted_time_windows_dict, tsp_H2_tour)
    print_and_store(f'Arriced in advance and WAIT, tour list = {tsp_H2_tour} and beta_H1 = {beta_H2} with delay: {delay}')
  elif not(sorted_time_windows_dict[target][0] <= beta_H2+temp[0]):
    print_and_store('Too early, go to nwxt costumer is queue')
    T = T +1
    if T >= len(sorted_time_windows_dict):
      delta = delta + delta//2
      T = 0
  else:
    print_and_store('Infeasible situation encountered.------------------------------------------------------------------')
    status_H2 = 0
    break

  print_and_store('\n-----------------------------------------------------------------------------------------------')

beta_H2 = beta_H2 + cost_matrix[tsp_H2_tour[-1]][0]
tsp_H2_tour.append(0)

toc = time.perf_counter()
tsp_H2_time = toc-tic

print_and_store('\nEND\n')

save_optimization_results(number_customers=N,
                          name_heuristic='Heuristic without A*',
                          status=status_H2,
                          objective_value=beta_H2,
                          computing_time=tsp_H2_time,
                          messages=messages)

if status_H2:
  print(f'Status: {status_H2}')
else:
  print(("Infisible, result saved in txt file"))
  raise ValueError("Infisible, result saved in txt file")  #Code Blocking


Actual max delay: 2
Actual T = 0
Actial beta = 0
Actual tour list = [0]
Costumer to visit ordered: [4, 3, 1, 2, 5]

Journet from 0 to 4

Direct path pass throw [0, 4], at cost: 4

beta_H2 at position 0 was 0 and beta_H2 at position 4 is 4
Time windows: ai = 23, bi = 44
Too early, go to nwxt costumer is queue

-----------------------------------------------------------------------------------------------

Actual max delay: 2
Actual T = 1
Actial beta = 0
Actual tour list = [0]
Costumer to visit ordered: [4, 3, 1, 2, 5]

Journet from 0 to 3

Direct path pass throw [0, 3], at cost: 7

beta_H2 at position 0 was 0 and beta_H2 at position 3 is 7
Time windows: ai = 16, bi = 64
Too early, go to nwxt costumer is queue

-----------------------------------------------------------------------------------------------

Actual max delay: 2
Actual T = 2
Actial beta = 0
Actual tour list = [0]
Costumer to visit ordered: [4, 3, 1, 2, 5]

Journet from 0 to 1

Direct path pass throw [0, 1], at cost: 9

bet

# Collection of results

In [ ]:
messages = []
status_lp = tsptw.status

print(f'Number of customes: {N}\n')

print_and_store(f'Pulp results:\n')

if status_lp:
  print_and_store(f"objective value: {tsp_pulp_value}")
  print_and_store(f"calculated in {round(tsp_pulp_time,3)} seconds")
  print_and_store(f"TSP tour: {tsp_pulp_tour}")
else:
  print_and_store(f'Rusult take too long, over 2 hours.')

print_and_store('\n')
#----------------------------------------------------------------------

print_and_store(f'Heuristic with A* results:\n')

if status_H1:
  print_and_store(f"objective value: {beta_H1}")
  print_and_store(f"calculated in {round(tsp_H1_time,3)} seconds")
  print_and_store(f'Total number of iterative cycle: {all_times}, A* is used {a_star_times} times and direct path is used {direct_times}')

  # print_and_store tour
  to_print =''
  for i in tsp_H1_tour:
    to_print = to_print + str(i) + ' -> '
  to_print = to_print + "Finished!"
  print_and_store(f'TSP tour: {to_print}')

else:
  print_and_store(f'Rusult is infisible')

print_and_store('\n')
#----------------------------------------------------------------------

print_and_store('Heuristic pure (without A*) results:\n')

if status_H2:
  print_and_store(f"objective value:  {beta_H2}")
  print_and_store(f'calculated in {round(tsp_H2_time,3)} seconds')
  print_and_store(f'Number of iterative cycle {counter_without_a}')

  # Print tour
  to_print =''
  for i in tsp_H2_tour:
    to_print = to_print + str(i) + ' -> '
  to_print = to_print + "Finished!"
  print_and_store(f"TSP tour: {to_print}")
else:
  print_and_store(f'Rusult is infisible')

Number of customes: 5

Pulp results:

objective value: 20.0
calculated in 0.052 seconds
TSP tour: 0 -> 4 -> 3 -> 2 -> 1 -> 5 -> 0 -> Finished!


Heuristic with A* results:

objective value: 37
calculated in 0.013 seconds
Total number of iterative cycle: 8, A* is used 0 times and direct path is used 8
TSP tour: 0 -> 1 -> 3 -> 4 -> 2 -> 5 -> 0 -> Finished!


Heuristic pure (without A*) results:

objective value:  37
calculated in 0.02 seconds
Number of iterative cycle 8
TSP tour: 0 -> 1 -> 3 -> 4 -> 2 -> 5 -> 0 -> Finished!


In [ ]:
def calculate_and_print_and_store_comparison(a_value, b_value, a_time, b_time, name_a, name_b):
    value_difference = a_value - b_value
    time_difference = a_time - b_time

    percent_value_difference = (value_difference / b_value) * 100
    percent_time_difference = (time_difference / b_time) * 100

    if value_difference > 0:
        print_and_store(f'{name_a} has a higher value by {round(abs(percent_value_difference), 1)}% compared to {name_b} objective value.')
    else:
        print_and_store(f'{name_a} has a lower value by {round(abs(percent_value_difference), 1)}% compared to {name_b} objective value.')

    if time_difference < 0:
        print_and_store(f'{name_a} is faster by {round(abs(percent_time_difference), 1)}% compared to {name_b} time.')
    else:
        print_and_store(f'{name_a} is slower by {round(abs(percent_time_difference), 1)}% compared to {name_b} time.')

if status_lp or (status_H1 and status_H2):
  print_and_store('\n\n------------------Comparative results------------------ \n')

  if status_H1 and status_lp:
    # Comparisons between Pulp and H1
    calculate_and_print_and_store_comparison(tsp_pulp_value, beta_H1, tsp_pulp_time, tsp_H1_time, 'Pulp', 'Heuristic A* H1')

    print_and_store('\n')

  if status_H1 and status_H2:
    # Comparisons between H1 and H2
    calculate_and_print_and_store_comparison(beta_H1, beta_H2, tsp_H1_time, tsp_H2_time, 'Heuristic A* H1', 'Heuristic H2')

    usage_of_a_start = round(((a_star_times)/all_times) *100,2)
    usage_vs_unusage_of_a_start =  round(((a_star_times)/counter_without_a) *100,2)

    print_and_store(f'A* path is prefered {usage_of_a_start}% times comparated of direct path')
    print_and_store(f'Adding a A* heuristic reduce {usage_vs_unusage_of_a_start}% the number of cycle than not using it')

    print_and_store('\n')

  if status_H2 and status_lp:
    # Comparisons between Pulp and H2
    calculate_and_print_and_store_comparison(tsp_pulp_value, beta_H2, tsp_pulp_time, tsp_H2_time, 'Pulp', 'Heuristic H2')




------------------Comparative results------------------ 

Pulp has a lower value by 45.9% compared to Heuristic A* H1 objective value.
Pulp is slower by 291.3% compared to Heuristic A* H1 time.


Heuristic A* H1 has a lower value by 0.0% compared to Heuristic H2 objective value.
Heuristic A* H1 is faster by 32.8% compared to Heuristic H2 time.
A* path is prefered 0.0% times comparated of direct path
Adding a A* heuristic reduce 0.0% the number of cycle than not using it


Pulp has a lower value by 45.9% compared to Heuristic H2 objective value.
Pulp is slower by 162.9% compared to Heuristic H2 time.


In [ ]:
# Save results into a file
def save_results(number_customers, name, messages):
    variables = "\n".join(messages)

    results_str = (
        f"Instance Name: {name}\n"
        f"Number of Clients: {number_customers}\n"
        f"\n{variables}"
    )

    file_name = f"{name}_{number_customers}.txt"

    with open(file_name, 'w') as file:
        file.write(results_str)

save_results(number_customers=N,
             name='Comparation_results',
             messages=messages)